# chapter 7. 분류3 결정트리(Decision Tree)

# part 1. 결정트리란?
질문(조건식)을 던져 데이터를 계속 나누는 분류 모델이다.\
비선형 경계도 만들 수 있고, 전처리(스케일링)에 덜 민감하다.\
대신 깊이가 깊어지면 훈련 데이터에 과하게 맞는 과적합이 쉽게 생긴다.

사람의 의사결정처럼 "이 조건이면 왼쪽, 아니면 오른쪽"을 반복한다.\
각 분기에서 "어떻게 나누면 클래스가 더 깔끔하게 섞이지 않게 될까"를 기준으로 질문을 고른다.

간단예시\
승객 나이 < 10 인가?\
요금 > 30 인가?\
위 질문을 여러 번 이어서 최종적으로 클래스를 결정한다.


### 트리모델의 핵심 용어
결정트리는 그림으로 보면 **나무**처럼 생겼고, 용어도 그 구조를 그대로 부른다.
- 노드(Node): 질문이 있는 점(분기점)
- 루트 노드(Root): 트리의 시작점(첫 질문)
- split(분기): 조건에 따라 좌/우로 나뉨
- 리프 노드(Leaf): 더 이상 질문하지 않고 최종 예측을 내는 끝점
- 깊이(Depth): 질문을 몇 번 거쳐 왔는지(트리가 얼마나 길게 내려갔는지)


## part 2. 불순도(impurity)와 분할 기준
불순도는 "한 노드 안에 클래스가 얼마나 섞여 있는가"를 수치로 표현한다.\
트리는 **불순도가 많이 줄어드는** 질문(분할)을 선택한다.

### 불순도의 대표 기준
- gini(지니): 섞여 있을수록 크고, 한 클래스가 대부분이면 작다.
- entropy(엔트로피): 불확실성이 클수록 크다.
- 두 기준 모두 **한 번 질문했을 때, 자식 노드들이 깔끔해지게 만들고 싶어한다.**

### 간단 예시 (지니로 감 잡기)
어떤 노드에 데이터 10개가 있고 라벨이 0/1이 5개씩 있다고 하자.\
→ 섞임이 심하니까 불순도가 높다.

Gini 불순도\
부모 노드(5:5) 지니: $1−(0.5^2+0.5^2)=0.5$

- 질문 A로 나눴더니
    - 왼쪽: (0만 5개) → 지니 0
    - 오른쪽: (1만 5개) → 지니 0
    - 가중평균 0 → 불순도 감소량 = 0.5 - 0 = 0.5 (엄청 좋음)

- 질문 B로 나눴더니
    - 왼쪽: (0이 4, 1이 1) → 지니  $1−(0.8^2+0.2^2)=0.32$
    - 오른쪽: (0이 1, 1이 4) → 지니 0.32
    - 가중평균 0.32 → 불순도 감소량 = 0.5 - 0.32 = 0.18 (A보다 별로)

→ 이렇게 되면 트리는 질문 A를 고른다. 더 "깔끔하게" 나누기 때문

## part 3. Iris 데이터로 결정트리 연습하기
Iris 데이터를 결정트리로 분류하고 기본 성능을 확인한다.

1) 데이터 불러오기  
2) train/test split  
3) DecisionTreeClassifier 학습  
4) accuracy, classification_report 확인

In [5]:
import numpy as np, pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import graphviz
from IPython.display import display

# 1) 데이터 불러오기  
iris = load_iris()
X, y = iris.data, iris.target
feature_names = iris.feature_names
target_names  = iris.target_names

# 2) train/test split  
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    stratify=y,  # train/test에서 클래스 비율 유지
    random_state=42 # 재현성 고정
)

# 3) DecisionTreeClassifier 학습  
# 결정 트리(max_depth=None == 깊이 제한 없음)
dt = DecisionTreeClassifier(max_depth=None, random_state=42)
dt.fit(X_train, y_train)

# 4) accuracy, classification_report 확인
y_pred = dt.predict(X_test)
# 전체 정답 비율 (정확도)
print("Accuracy:", accuracy_score(y_test, y_pred))
print()
# 클래스별 precision/recall/f1-score, support(표본 수)
print(classification_report(y_test, y_pred, target_names=target_names, digits=3)) 

Accuracy: 0.9333333333333333

              precision    recall  f1-score   support

      setosa      1.000     1.000     1.000        10
  versicolor      0.900     0.900     0.900        10
   virginica      0.900     0.900     0.900        10

    accuracy                          0.933        30
   macro avg      0.933     0.933     0.933        30
weighted avg      0.933     0.933     0.933        30



결정트리(max_depth=제한 없음)는 테스트셋에서 
- accuracy가 0.933...이며, 

- 클래스의 precision, recall, f1-score이 높게 나와 분류가 잘 되었다 볼수 있다.

어떤 클래스가 서로 헷갈리는지 혼동행렬로 확인한다.

In [6]:
cm = confusion_matrix(y_test, y_pred)
fig = ff.create_annotated_heatmap(
    z=cm, x=list(target_names), y=list(target_names),
    colorscale="Blues", showscale=True
)
fig.update_layout(title="Confusion Matrix (Decision Tree, max_depth=None)",
                  xaxis=dict(title="Predicted"), yaxis=dict(title="Actual"),
                  template="plotly_dark")
fig.show()

- 대각선: 맞춘 개수
    - setosa: 10개가 전부 setosa로 예측(10/10)
    - versicolor: 9개 정분류(9/10)
    - virginica: 9개 정분류(9/10)

- 비대각선: 틀린 개수(오분류)
    - 실제 versicolor 10개 중 1개가 virginica로 예측
    - 실제 virginica 10개 중 1개가 versicolor로 예측
    - 그 외(setosa 관련)는 오분류 0

혼동행렬에서...\
setosa는 오분류가 없지만, \
versicolor와 virginica 사이에 오분류가 각각 1건씩 발생했다.\
두 클래스를 구분하는 경계가 일부 겹치는 것으로 보인다.

## part 4. max_depth와 과적합
max_depth는 **질문 횟수 제한**이다.\
결정트리는 데이터를 분류할 때 질문을 연달아 하는데 그 연속 질문의 최대 횟수를 제한하는 값이다.\
값을 크게 하면 더 세밀하게 나누고, 너무 크게 하면 훈련 데이터를 외워 과적합이 생기기 쉽다.\
때문에 되도록이면 깊이를 제한해야 한다.

### max_depth와 예시
1. max_depth = 1 (최대 질문 1번 가능)\
구조: Q1: A > 3인가?
예 → 결론 | 아니오 → 결론\
즉, 한 번만 물어보고 바로 결정.

2. max_depth = 2 (최대 질문 2번 가능)\
구조: Q1\
예 → Q2 → 결론\
아니오 → Q2 → 결론\
즉, 최대 두 번까지 꼬리 질문 가능.

### max_depth = None이면 왜 위험한가?
결정트리는 **질문을 던져 데이터를 분류하는 모델**이다.\
max_depth=None이면 질문을 멈추지 않고 계속 쪼갤 수 있다.\
그러면 트리가가 데이터가 조금이라도 섞여 있으면 더 쪼개서 결국 거의 한 샘플 단위까지 맞춰버릴 수 있다.\
이렇게 되면 **과적합**이 발생해 모델의 성능이 떨어진다.

> max_depth의 깊이를 제한하면 과적합을 줄이고 일반화 성능을 올릴 수 있다.

## Part 5. 가지치기(Pruning) — 과적합을 막는 3가지 하이퍼파라미터
결정트리는 질문을 계속 늘리면 훈련 데이터를 너무 세밀하게 외워(과적합) 문제가 생긴다.\
가지치기(pruning)는 트리가 필요 이상으로 복잡해지지 않게 제한해, **일반화 성능(test 성능)**을 올리는 방법이다.

### 3가지 도구
1) max_depth — 질문 횟수 제한\
트리가 최대 몇 번까지 질문할 수 있는지 제한한다.

2) min_samples_split — 노드를 나누려면 최소 표본 수가 필요\
데이터가 너무 적은 곳에서 "쪼개기"를 못 하게 막는다

3) min_samples_leaf — 리프(끝 노드)에 최소 표본 수를 강제
1~2개 샘플만 맞추는 "외우기 리프"가 생기는 걸 방지\

| 파라미터 | 의미 | 효과 |
|---------|------|------|
| **max_depth** | 트리 최대 깊이 | 질문 횟수 제한 → 가장 강력한 가지치기 |
| **min_samples_split** | 분할에 필요한 **최소** 샘플 수 | 작은 노드는 더 쪼개지 않음 |
| **min_samples_leaf** | 잎 노드 **최소** 샘플 수 | 너무 작은 잎 노드 방지 |


> split/leaf 제약은 트리를 덜 예민하게 만들어 일반화를 돕는다.

### 코드 해설
depth만 제한한 경우와 split/leaf까지 제한한 경우를 비교한다.

In [14]:
# DecisionTreeClassifier 학습  
dt_depth4 = DecisionTreeClassifier(max_depth=4, random_state=42)
dt_depth4.fit(X_train, y_train)

dt_pruned = DecisionTreeClassifier(
    max_depth=4,                    # 깊이 4로 제한
    min_samples_split=8,            # 8개 미만이면 분기 금지
    min_samples_leaf=4,             # 리프에 최소 4개는 남겨야 함
    random_state=42
)
dt_pruned.fit(X_train, y_train)

print(f"[depth=4 only]         Train: {accuracy_score(y_train, dt_depth4.predict(X_train)):.3f} | "
      f"Test: {accuracy_score(y_test, dt_depth4.predict(X_test)):.3f}")
print(f"[depth=4 + split/leaf] Train: {accuracy_score(y_train, dt_pruned.predict(X_train)):.3f} | "
      f"Test: {accuracy_score(y_test, dt_pruned.predict(X_test)):.3f}")

[depth=4 only]         Train: 0.992 | Test: 0.933
[depth=4 + split/leaf] Train: 0.983 | Test: 0.967


## part 6. feature importance(특성 중요도)
모델이 예측할 때 어떤 변수(feature)를 더 많이 참고했는지를 숫자로 보여주는 값이다.

결정트리에서의 의미:\
트리가 분기에 얼마나 기여했는지를 기준으로 피처 중요도를 계산한다.\
이때 값이 크다고 인과를 의미하진 않는다. (단지 분류에 많이 쓰였다는 의미)\
상관이 높은 피처가 있으면 중요도가 분산될 수 있다.


한 줄 예시\
"꽃잎 길이"로 나누면 분류가 확 깔끔해지고,\
"꽃잎 폭"은 거의 도움이 안 됐다면\
→ 꽃잎 길이 중요도 ↑, 꽃잎 폭 중요도 ↓

> 중요도가 높다고 해서 **원인(인과)**이라는 뜻은 아니다. (그냥 "예측에 많이 썼다"는 단서일 뿐이다.)

In [15]:
# Feature Importance 시각화 (깊이 무제한 모델(예제용))
importances = pd.Series(dt.feature_importances_, index=feature_names).sort_values()

fig = px.bar(importances, orientation="h",
            title="Feature Importances (Decision Tree, max_depth=None)",
            template="plotly_dark")
fig.update_layout(xaxis_title="Importance", yaxis_title="Feature",
                showlegend=False)
fig.show()


Feature importance를 보면...\
petal lenght (cm) 피처의 기여도가 가장 높아, 트리가 주로 이 피처를 기준으로 분기한것으로 확인된다.

## part 10. GridSearchCV로 최적 파라미터 찾기
하이퍼파라미터 후보를 격자로 만들고, 교차검증 평균 점수로 가장 좋은 조합을 고른다.\
결정트리는 과적합이 쉽기 떄문에, 튜닝할 때 깊이/가지치기 파라미터가 특히 중요하다.

- max_depth(깊이)
- min_samples_split, min_samples_leaf(가지치기)
- gini/entropy



In [16]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "max_depth": [2, 3, 4, None],
    "min_samples_split": [2, 4, 8],
    "min_samples_leaf": [1, 2, 4]
}

gs = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    refit=True
)
gs.fit(X_train, y_train)

print(f"최적 파라미터: {gs.best_params_}")
print(f"최적 CV Accuracy: {gs.best_score_:.4f}")


최적 파라미터: {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
최적 CV Accuracy: 0.9417


GridSearchCV 결과, 최적 파라미터는...
max_depth: 4\
min_samples_leaf: 1\
min_samples_split: 2\
이며\
5-fold 평균 정확도는 0.9417 이다.

In [17]:
cv_df = pd.DataFrame(gs.cv_results_)
cv_table = cv_df[[
    "param_max_depth", "param_min_samples_split", "param_min_samples_leaf",
    "mean_test_score", "std_test_score", "rank_test_score"
]].sort_values("rank_test_score").head(5)

cv_table.columns = ["max_depth", "min_samples_split", "min_samples_leaf",
                     "cv_mean_acc", "cv_std", "rank"]
display(cv_table)

,max_depth,min_samples_split,min_samples_leaf,cv_mean_acc,cv_std,rank
27,None,2,1,0.941667,0.020412,1
18,4,2,1,0.941667,0.020412,1
12,3,2,2,0.933333,0.020412,3
15,3,2,4,0.933333,0.020412,3
10,3,4,1,0.933333,0.020412,3


- cv_mean_acc: 5-Fold 교차검증 평균 정확도
- cv_std: 표준편차 (작을수록 안정적)
- rank: 순위 (1이 최고)


상위 5개 조합의 교차검증 평균 정확도는 0.9333~0.9417이다.\
표준편차(std)는 모두 0.0204로 동일해 안정성 차이는 크지 않다.\
평균 정확도가 가장 높은 조합(0.9417) 중에서는 과적합을 줄이기 위해선 max_depth=4 조합을 우선 선택,\
최종 결론은 홀드아웃 테스트로 한 번 더 확인한다


상위 5개 조합의 평균 정확도는 ... 범위이며, 표준편차가 작은 조합이 더 안정적이다

## part 7. 최적 트리로 홀드아웃 테스트 평가
교차검증으로 고른 설정은 마지막에 test set에서 한 번 더 확인한다.\
test 성능이 크게 떨어지지 않으면 일반화가 괜찮다고 볼 수 있다.

## part 8. 실무팁: 결측치와 클래스 불균형
결측치가 있으면 많은 모델이 바로 학습이 안 된다. 먼저 채우거나 제거해야 한다.\
클래스 불균형(예: 90:10)이면 accuracy만 보면 모델이 좋아 보이는 착시가 생긴다.
class_weight='balanced'는 소수 클래스를 더 중요하게 보도록 가중치를 준다.


## part 9. PR Curve로 불균형 데이터 평가
- 불균형 문제에서는 ROC보다 PR Curve가 더 민감하게 차이를 보여주는 경우가 많다.
- AP(Average Precision)는 PR Curve를 하나의 숫자로 요약한 값이며, 클수록 좋다.


## part 10. 로지스틱 회귀 vs KNN vs 결정트리(간단 비교)

- 로지스틱 회귀: 선형 경계, 확률 해석이 깔끔, 스케일링/전처리에 민감할 수 있다.
- KNN: 거리 기반, 스케일링 중요, 데이터가 커지면 예측이 느려질 수 있다.
- 결정트리: 비선형 가능, 해석/시각화 쉬움, 과적합 위험이 커서 제약과 튜닝이 중요하다.